### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
school_data_complete.dtypes

Student ID        int64
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
School ID         int64
type             object
size              int64
budget            int64
dtype: object

In [3]:
total_schools = school_data.loc[:,["school_name"]]
num_schools = total_schools.count()[0]
total_students = school_data_complete.loc[:,["Student ID"]]
num_students = total_students.count()[0]
total_students = school_data_complete["Student ID"].count()
total_budget = round(school_data_complete["budget"].sum(),2)
average_math_score = round(school_data_complete["math_score"].mean(),2)
average_reading_score = round(school_data_complete["reading_score"].mean(),2)
students_math_over70 = school_data_complete.loc[school_data_complete["math_score"]>= 70]
students_over70_math = students_math_over70.count()[0]
percent_over70_math = round((students_over70_math/ num_students)*100, 2)
students_reading_over70 = school_data_complete.loc[school_data_complete["reading_score"]>= 70]
students_over70_reading = students_reading_over70.count()[0]
percent_over70_reading = round((students_over70_reading /num_students)*100,2)
#combined_MRstudents_over70 = (students_over70_math + students_over70_reading)
#overall_passing = round((combined_MRstudents_over70 / num_students)*100, 2)
overall_passing = float(percent_over70_math+percent_over70_reading)/2

summary_data_frame = pd.DataFrame({
    "Number of Schools": [num_schools],
    "Number of Students": [num_students],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score], 
    "% Passing Math" : [percent_over70_math],
    "% Passing Reading" : [percent_over70_reading],
    "% Overall Passing" : [overall_passing]    
       
    })

summary_data_frame

,Number of Schools,Number of Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,82932329558,78.99,81.88,74.98,85.81,80.395


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [4]:
school_summary = school_data_complete.groupby(['school_name'])
school_names = school_data_complete["school_name"].sort_values().unique()
school_types = school_data.sort_values(by="school_name").type
school_total_students = list(school_summary["student_name"].count())
school_budget = list(school_summary["budget"].mean())
school_per_student_budget = [a/b for a,b in zip(school_budget,school_total_students)]
school_average_math_score = list(school_summary["math_score"].mean())
school_average_reading_score = list(school_summary["reading_score"].mean())

# Calculating passing percentages
school_summary_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby(['school_name'])
school_percent_math = [(a/b)*100 for a,b in zip(school_summary_math["math_score"].count(),school_total_students)]
school_summary_reading = school_data_complete[school_data_complete["reading_score"] >= 70].groupby(['school_name'])
school_percent_reading = [(a/b)*100 for a,b in zip(school_summary_reading["reading_score"].count(),school_total_students)]
school_overall_passing = [(a+b)/2 for a,b in zip(school_percent_math,school_percent_reading)]

# Compile all calculations into dataframe
school_summary_df = pd.DataFrame({"School Names":school_names,
                                  "School Type":school_types,
                                  "Total Students":school_total_students,
                                  "Total School Budget":school_budget,
                                  "Per Student Budget":school_per_student_budget,
                                  "Average Math Score":school_average_math_score,
                                  "Average Reading Score":school_average_reading_score,
                                  "% Passing Math":school_percent_math,
                                  "% Passing Reading":school_percent_reading,
                                  "Overall Passing Rate":school_overall_passing})



school_summary_df.head()


,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
7,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
13,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [5]:
top_performing_schools = school_summary_df.sort_values(["Overall Passing Rate"],ascending=False)
top_performing_schools.head(5)


,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
14,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [6]:
bottom_performing_schools = school_summary_df.sort_values(["Overall Passing Rate"],ascending=True)
bottom_performing_schools.head(5)

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
11,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
12,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
13,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
grade_nine_math = school_data_complete[(school_data_complete["grade"] == "9th")]
grade_ten_math = school_data_complete[(school_data_complete["grade"] == "10th")]
grade_eleven_math = school_data_complete[(school_data_complete["grade"] == "11th")]
grade_twelve_math = school_data_complete[(school_data_complete["grade"] == "12th")]

grade_nine_math_scores = grade_nine_math.groupby(["school_name"]).mean()["math_score"]
grade_ten_math_scores = grade_ten_math.groupby(["school_name"]).mean()["math_score"]
grade_eleven_math_scores = grade_eleven_math.groupby(["school_name"]).mean()["math_score"]
grade_twelve_math_scores = grade_twelve_math.groupby(["school_name"]).mean()["math_score"]

math_scores_by_grade = pd.DataFrame({"9th Grade": grade_nine_math_scores,
                                "10th Grade": grade_ten_math_scores, 
                                "11th Grade": grade_eleven_math_scores, 
                                "12th Grade": grade_twelve_math_scores
                               })


math_scores_by_grade.head()

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
grade_nine_reading = school_data_complete[(school_data_complete["grade"] == "9th")]
grade_ten_reading = school_data_complete[(school_data_complete["grade"] == "10th")]
grade_eleven_reading = school_data_complete[(school_data_complete["grade"] == "11th")]
grade_twelve_reading = school_data_complete[(school_data_complete["grade"] == "12th")]

grade_nine_reading_scores = grade_nine_reading.groupby(["school_name"]).mean()["reading_score"]
grade_ten_reading_scores = grade_ten_reading.groupby(["school_name"]).mean()["reading_score"]
grade_eleven_reading_scores = grade_eleven_reading.groupby(["school_name"]).mean()["reading_score"]
grade_twelve_reading_scores = grade_twelve_reading.groupby(["school_name"]).mean()["reading_score"]

math_scores_by_grade = pd.DataFrame({"9th Grade": grade_nine_reading_scores,
                                "10th Grade": grade_ten_reading_scores, 
                                "11th Grade": grade_eleven_reading_scores, 
                                "12th Grade": grade_twelve_reading_scores
                               })


math_scores_by_grade.head()

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
bins = [0, 584.9, 629.9, 644.9, 675.9]
bin_names = ["<584", "585-629", "630-644", "645-675"]

scores_by_spending = school_summary_df[["Average Math Score",
                                    "Average Reading Score",
                                    "% Passing Math",
                                    "% Passing Reading",
                                    "Overall Passing Rate"]]
scores_by_spending["Spending Summary"] = pd.cut(school_summary_df["Per Student Budget"], bins, labels=bin_names)
scores_by_spending = scores_by_spending.groupby(["Spending Summary"])
scores_by_spending.mean()

<ipython-input-9-28824cb4dcb5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scores_by_spending["Spending Summary"] = pd.cut(school_summary_df["Per Student Budget"], bins, labels=bin_names)


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Summary,,,,,
<584,83.455399,83.933814,93.460096,96.610877,95.035486
585-629,81.899826,83.155286,87.133538,92.718205,89.925871
630-644,78.518855,81.624473,73.484209,84.391793,78.938001
645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [10]:
bins = [0, 1000, 2000, 5000]
bin_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

scores_by_school_size = school_summary_df[["Average Math Score",
                                    "Average Reading Score",
                                    "% Passing Math",
                                    "% Passing Reading",
                                    "Overall Passing Rate"]]
scores_by_school_size["School Size Summary"] = pd.cut(school_summary_df["Total Students"], bins, labels=bin_names)
scores_by_school_size = scores_by_school_size.groupby(["School Size Summary"])
scores_by_school_size.mean()

<ipython-input-10-6a9213881aae>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scores_by_school_size["School Size Summary"] = pd.cut(school_summary_df["Total Students"], bins, labels=bin_names)


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size Summary,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type

In [11]:
scores_by_type = school_summary_df[["School Names",
                                  "School Type",
                                  "Average Math Score",
                                  "Average Reading Score",
                                  "% Passing Math",
                                  "% Passing Reading",
                                  "Overall Passing Rate"]]

scores_by_type = scores_by_type.groupby("School Type")
scores_by_type.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
